# Markowitz Portfolio Optimization

### Import Packages

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

import matplotlib.pyplot as plt

### Input List of Tickers

In [ ]:
ticker_list = ['AAPL','MSFT','GOOG','AMZN','TSLA','META']

### Extract Closing Price from YFinance and Compile into Data Frame

In [ ]:
count = 0
for ticker in ticker_list:
    if count == 0:
        ticker_y = yf.Ticker(ticker)
        df = ticker_y.history(period="5y")
        df = df.reset_index()
        df = df.loc[:, ('Date','Close')]
        df = df.rename({'Close':ticker},axis=1)
        count += 1
        
    else:
        ticker_y = yf.Ticker(ticker)
        df_append = ticker_y.history(period="5y")
        df_append = df_append.reset_index()
        df_append = df_append.loc[:, ('Date','Close')]
        df_append = df_append.rename({'Close':ticker},axis=1)
        if len(df_append)==len(df):
            df[ticker]= df_append[ticker]
        else:
            print(ticker + ' does not match the length beacuse the stock is too new remove it')

In [ ]:
df_tickers = df.drop('Date',axis=1)

In [ ]:
df_tickers.head()

### Find Daily and Annual Returns

In [ ]:
returns_daily = df_tickers.pct_change()
returns_annual = returns_daily.mean() * 250

In [ ]:
cov_daily = returns_daily.cov()
cov_annual = cov_daily * 250

### Create Portfolios Adjusting for Weights and Calculate Volatility, Returns and Sharpe Ratio

In [ ]:
returns_list = []
volatility_list = []
sharpe_ratio = []
stock_weights = []


num_assets = len(ticker_list)
num_portfolios = 200000


for single_portfolio in range(num_portfolios):
    weights = np.random.random(num_assets)
    weights /= np.sum(weights)
    returns = np.dot(weights, returns_annual)
    volatility = np.sqrt(np.dot(weights.T, np.dot(cov_annual, weights)))
    sharpe = returns/volatility
    sharpe_ratio.append(sharpe)
    returns_list.append(returns)
    volatility_list.append(volatility)
    stock_weights.append(weights)

portfolio = {'Returns': returns_list,
             'Volatility': volatility_list,
            'Sharpe_Ratio':sharpe_ratio}


for counter,symbol in enumerate(ticker_list):
    portfolio[symbol+'_weight'] = [weight[counter] for weight in stock_weights]


df = pd.DataFrame(portfolio)


column_order = ['Returns', 'Volatility','Sharpe_Ratio'] + [stock+'_weight' for stock in ticker_list]

df = df[column_order]

df.head()

In [ ]:
minimum_volatility = df.Volatility.min()
maximum_sharpe = df.Sharpe_Ratio.max()

In [ ]:
maximum_sharpe_portfolio = df.loc[df.Sharpe_Ratio == maximum_sharpe]
minimum_volatility_portfolio = df.loc[df.Volatility == minimum_volatility]

### Minimum Volatility Weightings

In [ ]:
minimum_volatility_portfolio.T

### Maximum Sharpe Ratio Weightings

In [ ]:
maximum_sharpe_portfolio.T

### Plot Results

In [ ]:
plt.rcParams.update({'figure.figsize':(20,13)})
plt.rcParams.update({'font.size':20})

In [ ]:
plt.scatter(df.Volatility, df.Returns,marker='.',c = df.Sharpe_Ratio, cmap='RdYlGn')
plt.colorbar().set_label('Sharpe Ratio')
plt.xlabel('Volatility')
plt.ylabel('Returns')
plt.title('Markowitz’s Efficient Frontier')

plt.scatter(x=maximum_sharpe_portfolio.Volatility, y=maximum_sharpe_portfolio.Returns, c='firebrick', marker='P', s=400, label="maximum_sharpe")
plt.scatter(x=minimum_volatility_portfolio.Volatility, y=minimum_volatility_portfolio.Returns, c='royalblue', marker='X', s=400, label="minimum_volatility")
plt.legend()